## Dependencies

In [1]:
!pip uninstall -y tensorflow numpy && pip install tensorflow-gpu==1.14.0 tf_sentencepiece

Uninstalling tensorflow-1.14.0:
  Successfully uninstalled tensorflow-1.14.0
Uninstalling numpy-1.16.5:
  Successfully uninstalled numpy-1.16.5
     |████████████████████████████████| 377.0MB 65kB/s 
     |████████████████████████████████| 2.7MB 42.2MB/s 
     |████████████████████████████████| 20.4MB 1.8MB/s 
ERROR: stable-baselines 2.2.1 requires tensorflow>=1.5.0, which is not installed.
ERROR: magenta 0.3.19 requires tensorflow>=1.12.0, which is not installed.
ERROR: fancyimpute 0.4.3 requires tensorflow, which is not installed.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [2]:
!curl -o one_to_many.zip https://dl.dropboxusercontent.com/s/bssvce8t00zjwxj/one_to_many.zip?dl=0

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  115M  100  115M    0     0  38.4M      0  0:00:03  0:00:03 --:--:-- 38.4M


In [0]:
!unzip -q one_to_many.zip

In [4]:
!curl -o mlt.zip https://codeload.github.com/suyash/mlt/zip/master

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 28981    0 28981    0     0   149k      0 --:--:-- --:--:-- --:--:--  150k


In [0]:
!unzip -q mlt.zip && mv mlt-master/mlt ./

## One to Many Translation

In [0]:
import numpy as np
import tensorflow as tf
import tf_sentencepiece as tfs

from mlt.evaluation import predict
from mlt.layers import Attention, ConditionalNormalization, Gelu, MultiplyConstant, PaddingAndLookaheadMask, PaddingMask, PositionalEncoding

In [7]:
tf.__version__

'1.14.0'

In [0]:
sess = tf.InteractiveSession()

In [0]:
en_model_file = "final/sentencepiece/para_crawl/ende_plain_text/models/unigram/8192/a.model"
de_model_file = "final/sentencepiece/para_crawl/ende_plain_text/models/unigram/8192/b.model"
fr_model_file = "final/sentencepiece/para_crawl/enfr_plain_text/models/unigram/8192/b.model"
es_model_file = "final/sentencepiece/para_crawl/enes_plain_text/models/unigram/8192/b.model"
it_model_file = "final/sentencepiece/para_crawl/enit_plain_text/models/unigram/8192/b.model"

In [0]:
en_offset = 0
fr_offset = 8192
de_offset = fr_offset + 8192
es_offset = de_offset + 8192
it_offset = es_offset + 8192

In [11]:
with sess.as_default():
    model = tf.keras.experimental.load_from_saved_model("final/model114", custom_objects={
        "MultiplyConstant": MultiplyConstant,
        "PositionalEncoding": PositionalEncoding,
        "PaddingMask": PaddingMask,
        "PaddingAndLookaheadMask": PaddingAndLookaheadMask,
        "Attention": Attention,
        "ConditionalNormalization": ConditionalNormalization,
        "Gelu": Gelu,
    })

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


### English to German

In [0]:
a = tfs.encode(["This is a problem that we have to solve."], model_file=en_model_file, add_bos=True, add_eos=True)[0] + en_offset

In [13]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([1.0]),
    tarf=tf.constant([1.0, 0.0, 0.0, 0.0]),
    bos_id=de_offset + 1,
    eos_id=de_offset + 2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
ids = ids + mask * -de_offset
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [16]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=de_model_file))], probs_[0]

(['Dies ist ein Problem, das wir lösen wollen.',
  'Das ist ein Problem, das wir lösen wollen.',
  'Dies ist ein Problem, das wir lösen können.',
  'Das ist ein Problem, das wir lösen können.',
  'Das ist ein Problem, das wir lösen haben.'],
 array([0.5739393 , 0.5204113 , 0.34543523, 0.31465307, 0.2541431 ],
       dtype=float32))

### English to French

In [0]:
a = tfs.encode(["This is a problem that we have to solve"], model_file=en_model_file, add_bos=True, add_eos=True)[0] + en_offset

In [0]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([1.0]),
    tarf=tf.constant([0.0, 1.0, 0.0, 0.0]),
    bos_id=fr_offset + 1,
    eos_id=fr_offset + 2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
ids = ids + mask * -fr_offset
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [21]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=fr_model_file))], probs_[0]

(['Ce problème est un problème qui nous faut résoudre.',
  'Ce problème est un problème qui nous asombant',
  'Ce problème est un problème qui nous faut résoudre',
  'Ce problème est un problème qui nous asombant.',
  "C'est un problème qui nous a pour résoudre"],
 array([0.29952338, 0.2466469 , 0.192374  , 0.17028809, 0.16088726],
       dtype=float32))

### English to Spanish

In [0]:
a = tfs.encode(["This is a problem that we have to solve"], model_file=en_model_file, add_bos=True, add_eos=True)[0] + en_offset

In [0]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([1.0]),
    tarf=tf.constant([0.0, 0.0, 1.0, 0.0]),
    bos_id=es_offset + 1,
    eos_id=es_offset + 2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
ids = ids + mask * -es_offset
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [26]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=es_model_file))], probs_[0]

(['Este es un problema que necesitamos resolver',
  'Este es un problema que nosotros necesitamos resolver',
  'Este es un problema que tenemos que vender',
  'Es un problema que necesitamos resolver',
  'Este es un problema que tenemos para solucionar'],
 array([0.63790417, 0.33507025, 0.2521149 , 0.2309758 , 0.20429674],
       dtype=float32))

### English to Italian

In [0]:
a = tfs.encode(["This is a problem that we have to solve"], model_file=en_model_file, add_bos=True, add_eos=True)[0] + en_offset

In [0]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([1.0]),
    tarf=tf.constant([0.0, 0.0, 0.0, 1.0]),
    bos_id=it_offset + 1,
    eos_id=it_offset + 2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
ids = ids + mask * -it_offset
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [31]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=it_model_file))], probs_[0]

(['Questo è un problema che abbiamo bisogno di risolvere',
  'Questo è un problema che vogliamo',
  'Questo è un problema che abbiamo dovuto risolvere',
  'Questo è un problema che vogliamo risolvere',
  'Questo è un problema che dobbiamo risolvere'],
 array([0.5477947 , 0.5022847 , 0.24332887, 0.22610326, 0.20580557],
       dtype=float32))

In [0]:
sess.close()